## 关于LCEL语法的使用

在使**用 $|$** 之前的举例:

In [ ]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
import os
import dotenv
from langchain_core.utils import pre_init
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="""回答用户的查询。\n满足的格式为{format_instructions}\n问题为{question}\n""",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# prompt = prompt_template.invoke(input={"question": joke_query})
# response = chat_model.invoke(prompt)
# print(response)

# json_result = parser.invoke(response)

chain = prompt_template|chat_model|parser
response = chain.invoke(input={"question" : joke_query})
print(response)


# 2、顺序链之SimpleSequentialChain的使用

举例1:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

chainA_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位精通各领域知识的知名教授"),
        ("human", "请你尽可能详细的解释一下: {knowledge}"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini")
# 假设 llm 已经是一个有效的语言模型实例
# 例如：from langchain_openai import ChatOpenAI; llm = ChatOpenAI(model="gpt-4o-mini")
# 请确保在运行代码时 llm 变量已定义。

chainA_chains = LLMChain(llm=llm,
                         prompt=chainA_template,
                         verbose=True
)

# # chainA_chains.invoke({"knowledge": "什么是LangChain？"})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain # Assuming LLMChain is imported from the previous snippet

chainB_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息, 并做出简短的总结"),
        ("human", "这是针对一个提问的完整的解释说明内容: {description}"),
        ("human", "请你根据上述说明, 尽可能简短的输出重要的结论, 请控制在20个字以内"),
    ]
)

# 假设 chat_model 已经是一个有效的语言模型实例
# 例如：from langchain_openai import ChatOpenAI; chat_model = ChatOpenAI(model="gpt-4o-mini")
# 请确保在运行代码时 chat_model 变量已定义。

chainB_chains = LLMChain(llm=chat_model,
                         prompt=chainB_template,
                         verbose=True
)

In [ ]:
from langchain.chains.sequential import SimpleSequentialChain # 假设 SimpleSequentialChain 已导入

full_chain = SimpleSequentialChain(
    chains = [chainA_chains, chainB_chains],
    verbose=True
)
# 说明: 针对于SimpleSequentialChain而言, 唯一的输入变量名是: input
# 就算前面叫Knowledge也只能写input
response = full_chain.invoke(input={"input" :"什么是LangChain"})

print(response)

In [ ]:
# 1.导入相关包
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
# 2.创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")
# 3.定义一个给剧名写大纲的LLMChain
template1 = """你是个剧作家。给定剧本的标题，你的工作就是为这个标题写一个大纲。
Title: {title}
"""
prompt_template1 = PromptTemplate(input_variables=["title"], template=template1)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template1)
# 4.定义给一个剧本大纲写一篇评论的LLMChain
template2 = """你是《纽约时报》的剧评家。有了剧本的大纲，你的工作就是为剧本写一篇评论
剧情大纲:
{synopsis}
"""
prompt_template2 = PromptTemplate(input_variables=["synopsis"], template=template2)
review_chain = LLMChain(llm=llm, prompt=prompt_template2)
# 5.定义一个完整的链按顺序运行这两条链
#(verbose=True:打印链的执行过程)
overall_chain = SimpleSequentialChain(
chains=[synopsis_chain, review_chain], verbose=True
) #
# 6.调用完整链顺序执行这两个链
review = overall_chain.invoke("日落海滩上的悲剧")
# 7.打印结果
print(review)

# 3、顺序链之SequentialChain的使用
举例1:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from openai import OpenAI
import os
# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")
schainA_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位精通各领域知识的知名教授"),
        ("human", "请你先尽可能详细的解释一下：{knowledge}，并且{action}")
    ]
)
schainA_chains = LLMChain(llm=llm,
    prompt=schainA_template,
    verbose=True,
    output_key="schainA_chains_key"
)


schainB_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
        ("human", "这是针对一个提问完整的解释说明内容：{schainA_chains_key}"),
        ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在100个字以内"),
    ]
)
schainB_chains = LLMChain(llm=llm,
    prompt=schainB_template,
    verbose=True,
    output_key='schainB_chains_key'
)

# 一定要声明两个变量: input_variables, output_variables
Seq_chain = SequentialChain(
    chains=[schainA_chains, schainB_chains],
    input_variables=["knowledge", "action"],
    output_variables=["schainA_chains_key","schainB_chains_key"],
    verbose=True)

response = Seq_chain.invoke({
    "knowledge":"中国足球为什么踢得烂",
    "action":"举一个实际的例子"
    })
print(response)

In [ ]:
# 可以单独打印
print(response["schainA_chains_key"])
print(response["schainB_chains_key"])

In [ ]:
# 1.导入相关包
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import SequentialChain

# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")
# 2.定义任务链一
#chain 1 任务：翻译成中文
first_prompt = PromptTemplate.from_template("把下面内容翻译成中文:\n\n{content}")
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=True,
    output_key="Chinese_Review",
)
#
# 3.定义任务链二
#chain 2 任务：对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = PromptTemplate.from_template("用一句话总结下面内容:\n\n{Chinese_Review}")
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    verbose=True,
    output_key="Chinese_Summary",
) #
# 4.定义任务链三
# chain 3 任务：识别语言
third_prompt = PromptTemplate.from_template("下面内容是什么语言:\n\n{Chinese_Summary}")
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    verbose=True,
    output_key="Language",
) #
# 5.定义任务链四
#chain 4 任务:针对摘要使用指定语言进行评论 input_key是上一个chain的output_key
fourth_prompt = PromptTemplate.from_template("请使用指定的语言对以下内容进行评论:\n\n内容:{Chinese_Summary}\n\n语言:{Language}")
chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    verbose=True,
    output_key="Comment",
)
#
# 6.总链
#overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言
# 进行评论

overall_chain = SequentialChain(
chains=[chain_one, chain_two, chain_three, chain_four],
verbose=True,
input_variables=["content"],
output_variables=["Chinese_Review", "Chinese_Summary", "Language", "Comment"],) #读取文件
# #read file
content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049- 45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492- 3850, email: michael.johnson@example.com)."
overall_chain.invoke(content)

# 1、create_sql_query_chain的使用
举例1:

In [ ]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# 测试连接本地的mysql数据库
db_user = "root"
db_password = "abc123"
db_host = "localhost" # 或 127.0.0.1
db_port = "3306"
db_database = "atguigudb"

# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

print("操作的是哪种数据库:", db.dialect)
print("获取数据库中的表:", db.get_usable_table_names())

# 执行查询操作
res = db.run("SELECT COUNT(*) FROM employees")
print(res)

举例2: chain的使用

In [ ]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# #1、获取mysql数据库的连接
# 测试连接本地的mysql数据库
db_user = "root"
db_password = "abc123"
db_host = "localhost" # 或 127.0.0.1
db_port = "3306"
db_database = "atguigudb"

# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

# 2、获取大语言模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(model="gpt-4o-mini")

# 3、创建create_sql_query_chain的实例
# Assumes 'create_sql_query_chain' and 'db' are imported/defined in the preceding code
chain = create_sql_query_chain(chat_model, db)
response = chain.invoke({"question": "数据表employees中薪资最高的员工信息",
                         "table_names_to_use": ["employees"]})
# input key必须叫question